In [1]:
import pandas as pd

RAW_PATH = "../DataSets/raw/risk_dataset_60000.csv"
df = pd.read_csv(RAW_PATH)

print(df.shape)
df.head()


(60000, 42)


,policy_id,customer_id,driver_age,driver_gender,driver_occupation,years_of_driving_experience,member_automobile_assoc_ceylon,has_previous_motor_policy,ncb_percentage,accidents_last_3_years,...,wc_paid_driver,cover_flood_damage,cover_strike_riot_civil_commotion,vehicle_age_years,vehicle_segment,approx_market_value,sum_insured,had_claim_within_1_year,num_claims_within_1_year,total_claim_amount_within_1_year
0,P000001,C00002,62,M,Businessman,4,0,1,35,0,...,0,1,0,4,Sedan,6633141,6788152,0,0,0
1,P000002,C00003,69,F,Businessman,20,0,1,20,0,...,1,0,0,23,Van,9024775,7293907,0,0,0
2,P000003,C00004,37,M,IT Engineer,13,0,0,0,0,...,1,0,0,25,Sedan,3697614,3128141,0,0,0
3,P000004,C00005,63,F,Businessman,31,0,1,20,1,...,0,0,0,15,Van,6208884,5898440,0,0,0
4,P000005,C00006,21,M,Student,0,0,0,0,1,...,0,1,0,18,SmallCar,4297644,3796591,0,0,0


In [10]:
# Shape
print("Dataset shape (rows, columns):", df.shape)

# Column types & non-null counts
df.info()


Dataset shape (rows, columns): (60000, 44)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 44 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   policy_id                          60000 non-null  object  
 1   customer_id                        60000 non-null  object  
 2   driver_age                         60000 non-null  float64 
 3   driver_gender                      60000 non-null  object  
 4   driver_occupation                  60000 non-null  object  
 5   years_of_driving_experience        60000 non-null  float64 
 6   member_automobile_assoc_ceylon     60000 non-null  int64   
 7   has_previous_motor_policy          60000 non-null  int64   
 8   ncb_percentage                     60000 non-null  int64   
 9   accidents_last_3_years             60000 non-null  int64   
 10  vehicle_type                       60000 non-null  object  
 11

In [11]:
df_clean = df.copy()


# Document-related fields: missing = 0
document_cols = [
    "images_uploaded",
    "inspection_report_uploaded",
    "registration_book_available"
]

for col in document_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna(0).astype(int)

# Categorical fields: missing = "Unknown"
cat_cols = df_clean.select_dtypes(include="object").columns
df_clean[cat_cols] = df_clean[cat_cols].fillna("Unknown")

In [4]:
def clip_invalid(col, low=None, high=None):
    if col in df.columns:
        if low is not None:
            df.loc[df[col] < low, col] = pd.NA
        if high is not None:
            df.loc[df[col] > high, col] = pd.NA

clip_invalid("driver_age", 18, 90)
clip_invalid("years_of_driving_experience", 0, 80)
clip_invalid("vehicle_age_years", 0, 60)
clip_invalid("engine_capacity_cc", 200, 10000)

print("Invalids handled. Missing% top 10:")
display((df.isna().mean()*100).sort_values(ascending=False).head(10))


Invalids handled. Missing% top 10:


policy_id                         0.0
customer_id                       0.0
driver_age                        0.0
driver_gender                     0.0
driver_occupation                 0.0
years_of_driving_experience       0.0
member_automobile_assoc_ceylon    0.0
has_previous_motor_policy         0.0
ncb_percentage                    0.0
accidents_last_3_years            0.0
dtype: float64

In [12]:
from datetime import datetime

current_year = datetime.now().year

df_clean = df_clean[
    (df_clean["driver_age"].between(18, 85)) &
    (df_clean["engine_capacity_cc"].between(600, 5000)) &
    (df_clean["vehicle_year_of_manufacture"].between(1985, current_year)) &
    (df_clean["ncb_percentage"].between(0, 65))
]



In [13]:
# Driving experience rule
df_clean = df_clean[
    df_clean["years_of_driving_experience"] <= (df_clean["driver_age"] - 18)
]

# Vehicle age derivation
df_clean["vehicle_age_years"] = current_year - df_clean["vehicle_year_of_manufacture"]

# NCB logic
df_clean.loc[
    df_clean["has_previous_motor_policy"] == 0,
    "ncb_percentage"
] = 0

# Blacklisted customer hard flag
df_clean["hard_flag_blacklist"] = (
    df_clean["is_blacklisted_customer"] == 1
).astype(int)

In [14]:
df_fe = df_clean.copy()

df_fe["driver_age_band"] = pd.cut(
    df_fe["driver_age"],
    bins=[17, 24, 34, 44, 59, 120],
    labels=["18–24", "25–34", "35–44", "45–59", "60+"]
)


In [15]:
df_fe["vehicle_age_band"] = pd.cut(
    df_fe["vehicle_age_years"],
    bins=[-1, 3, 7, 12, 100],
    labels=["0–3", "4–7", "8–12", "13+"]
)


In [16]:
df_fe[
    ["driver_age", "driver_age_band",
     "vehicle_age_years", "vehicle_age_band"]
].head(10)


,driver_age,driver_age_band,vehicle_age_years,vehicle_age_band
0,62.0,60+,4,4–7
1,69.0,60+,23,13+
2,37.0,35–44,25,13+
3,63.0,60+,15,13+
4,21.0,18–24,18,13+
5,50.0,45–59,17,13+
6,35.0,35–44,3,0–3
7,36.0,35–44,5,4–7
8,56.0,45–59,10,8–12
9,21.0,18–24,18,13+


In [20]:
import numpy as np

df_fe["risk_exposure_proxy"] = np.where(
    df_fe["vehicle_usage_type"]
        .astype(str)
        .str.strip()
        .str.lower()
        .isin([
            "commercial",
            "hire",
            "rent",
            "goods transport",
            "passenger hire"
        ]),
    "High",
    "Low"
)


In [21]:
doc_cols = [
    "images_uploaded",
    "inspection_report_uploaded",
    "registration_book_available"
]

df_fe["doc_missing_score"] = (
    (df_fe[doc_cols] == 0).sum(axis=1)
)


In [22]:
df_fe["compliance_risk_score"] = (
    ((df_fe["rebate_offered"] == 1) & (df_fe["rebate_within_company_limits"] == 0)).astype(int)
    + (df_fe["registration_book_available"] == 0).astype(int)
    + (df_fe["inspection_report_uploaded"] == 0).astype(int)
)


In [23]:
df_fe["ncb_validity_flag"] = np.where(
    (df_fe["has_previous_motor_policy"] == 0) &
    (df_fe["ncb_percentage"] > 0),
    1,
    0
)


In [24]:
df_fe[
    [
        "driver_age", "driver_age_band",
        "vehicle_age_years", "vehicle_age_band",
        "vehicle_usage_type", "risk_exposure_proxy",
        "doc_missing_score",
        "compliance_risk_score",
        "ncb_validity_flag"
    ]
].head(10)


,driver_age,driver_age_band,vehicle_age_years,vehicle_age_band,vehicle_usage_type,risk_exposure_proxy,doc_missing_score,compliance_risk_score,ncb_validity_flag
0,62.0,60+,4,4–7,Private,Low,0,0,0
1,69.0,60+,23,13+,Private,Low,0,0,0
2,37.0,35–44,25,13+,Private,Low,0,0,0
3,63.0,60+,15,13+,Hire,High,1,0,0
4,21.0,18–24,18,13+,Private,Low,0,0,0
5,50.0,45–59,17,13+,Private,Low,0,0,0
6,35.0,35–44,3,0–3,Private,Low,1,1,0
7,36.0,35–44,5,4–7,Private,Low,0,0,0
8,56.0,45–59,10,8–12,Commercial,High,0,1,0
9,21.0,18–24,18,13+,Hire,High,0,0,0


In [25]:
CLEAN_PATH = "../DataSets/processed/risk_dataset_clean.csv"
df.to_csv(CLEAN_PATH, index=False)
print("Saved:", CLEAN_PATH, "| Shape:", df.shape)


Saved: ../DataSets/processed/risk_dataset_clean.csv | Shape: (60000, 44)


In [26]:
# -----------------------------
# FINAL MODELING DATASET (RISK)
# -----------------------------

RISK_FEATURES = [
    "driver_age", "driver_age_band", "driver_gender", "driver_occupation",
    "years_of_driving_experience", "member_automobile_assoc_ceylon",
    "has_previous_motor_policy", "accidents_last_3_years", "ncb_percentage",
    "vehicle_type", "vehicle_segment", "engine_capacity_cc", "fuel_type",
    "vehicle_age_years", "vehicle_age_band", "has_lpg_conversion",
    "vehicle_usage_type", "registration_district",
    "parking_type"
]

TARGET = "had_claim_within_1_year"

# sanity checks
missing_cols = [c for c in RISK_FEATURES + [TARGET] if c not in df.columns]
assert len(missing_cols) == 0, f"Missing columns: {missing_cols}"

df_model = df[RISK_FEATURES + [TARGET]].copy()

MODEL_PATH = "../DataSets/processed/risk_dataset_model.csv"
df_model.to_csv(MODEL_PATH, index=False)

print("Saved modeling dataset:", MODEL_PATH)
print("Shape:", df_model.shape)


Saved modeling dataset: ../DataSets/processed/risk_dataset_model.csv
Shape: (60000, 20)
